%pip list

In [1]:
import requests
from bs4 import BeautifulSoup

import dotenv
import os

# Load envorinment variables from .env file located in ../
dotenv.load_dotenv("../.env")

True

In [2]:
api_key = os.getenv("OPEN_AI_API_KEY")

In [3]:
import openai


class RecipeExtractor:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key

    def extract_recipe_info(self, url):
        # Step 1: Scrape the webpage
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {url}")

        soup = BeautifulSoup(response.content, "html.parser")

        # Step 2: Extract raw content for the LLM to process
        title = (
            soup.find("h1").get_text(strip=True)
            if soup.find("h1")
            else "No title found"
        )
        raw_text = " ".join([p.get_text(strip=True) for p in soup.find_all("p")])

        print(raw_text)
        # Step 3: Use OpenAI for processing
        # Generate a short summary of the recipe
        summary_prompt = f"Here is some text from a recipe webpage: \n\n{raw_text}\n\nPlease provide a short summary of this recipe."
        summary_response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": summary_prompt},
            ],
            max_tokens=50,
        )
        # print(summary_response['choices'][0]['message'])
        summary = summary_response.choices[0].message.content
        # Extract the list of ingredients
        ingredients_prompt = f"Here is some text from a recipe webpage: \n\n{raw_text}\n\nPlease provide only the names of the ingredients used in this recipe separated by commas."
        ingredients_response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": ingredients_prompt},
            ],
            max_tokens=100,
        )
        ingredients = ingredients_response.choices[0].message.content.split(",")
        ingredients = [ingredient.strip() for ingredient in ingredients]
        return {"title": title, "summary": summary, "ingredients": ingredients}

In [48]:
# Test with the provided URL
url = "https://www.marmiton.org/recettes/recette_cinnamon-rolls-de-karine_327467.aspx"
extractor = RecipeExtractor(api_key)
recipe_info = extractor.extract_recipe_info(url)

# Display the extracted information
print("Title:", recipe_info["title"])
print("Summary:", recipe_info["summary"])
print("Ingredients:", recipe_info["ingredients"])

Mieux manger Les Jours Les Plus Food Petit-déj gourmand Actus Accueil>Recettes>Cinnamon rolls de Karine Dissoudre la levure dans le lait tiède. Dans le bol de votre robot - ou dans un saladier si vous le faites à la main - verser la levure et ajouter la farine, le sel, le sucre et l'oeuf ; pétrir pendant 10 minutes. Ajoutez le beurre en morceaux et pétrir à nouveau 10 minutes. Sorter la pâte du robot et la mettre à lever dans un saladier couvert d'un torchon pendant 1h30. Entre temps, la pâte doit avoir doublé de volume. Pendant ce temps, préparer le mélange à la cannelle en mélangeant dans un bol le beurre fondu, le sucre roux et la cannelle. La retirer du bol et l'étaler sur un plan de travail fariné en formant un rectangle. Étaler le mélange à la cannelle sur toute la surface du rectangle. Former un boudin avec la pâte en la roulant sur elle-même. Découper le boudin en tranches. Vous pouvez faire des cinnamons individuels en déposant les tranches dans des moules à muffins par exempl

In [35]:
raw_text = "Mieux manger Les Jours Les Plus Food Petit-déj gourmand Actus Accueil>Recettes>Cinnamon rolls de Karine Dissoudre la levure dans le lait tiède. Dans le bol de votre robot - ou dans un saladier si vous le faites à la main - verser la levure et ajouter la farine, le sel, le sucre et l'oeuf ; pétrir pendant 10 minutes. Ajoutez le beurre en morceaux et pétrir à nouveau 10 minutes. Sorter la pâte du robot et la mettre à lever dans un saladier couvert d'un torchon pendant 1h30. Entre temps, la pâte doit avoir doublé de volume. Pendant ce temps, préparer le mélange à la cannelle en mélangeant dans un bol le beurre fondu, le sucre roux et la cannelle. La retirer du bol et l'étaler sur un plan de travail fariné en formant un rectangle. Étaler le mélange à la cannelle sur toute la surface du rectangle. Former un boudin avec la pâte en la roulant sur elle-même. Découper le boudin en tranches. Vous pouvez faire des cinnamons individuels en déposant les tranches dans des moules à muffins par exemple ou en faire une grande brioche en disposant les tranches côte à côte sans trop les coller (elles vont encore gonfler) dans un moule rond de 24 cm environ. Recouvrir la pâte d'un linge propre et la remettre à lever entre 45 minutes et 1 heure. Préchauffer le four à 180°C (thermostat 6). Enfourner les cinnamons rolls pendant 25 minutes. Laisser refroidir. Pendant ce temps, préparer le glaçage en mélangeant le sucre glace et le blanc d'oeuf. Quand les gâteaux ont bien refroidi, verser le glaçage en filets à l'aide d'une fourchette ou d'une cuillère. Incrrrrrrr >>> l'une des meilleures recettes de cinnamon rolls vrm Cette recette est excellente, je n’avais jamais mangé une aussi bonne brioche à la cannelle ! Lire tous les commentaires Plus de recettes Ces contenus devraient vous intéresser  Concocté avec 🧡 par Marmiton Tous droits réservés Marmiton.org - 1999-2024 Paramétrer les cookies●Recrutement●Mentions légales●Conditions Générales d'Utilisation●Conditions Générales d'Abonnement●Contact●FAQ●Politique de confidentialité●Foire aux questions - Vos choix concernant l'utilisation de cookies Pour des milliers de cocktails :1001cocktails.com"

In [36]:
summary_prompt = f"Here is some text from a recipe webpage: \n\n{raw_text}\n\nPlease provide a short summary of this recipe."
summary_response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": summary_prompt},
    ],
    max_tokens=50,
)
print(summary_response["choices"][0]["message"])

TypeError: 'ChatCompletion' object is not subscriptable

In [45]:
summary_response.choices[0].message.content

'This recipe is for cinnamon rolls. It involves dissolving yeast in warm milk, mixing it with flour, sugar, salt, egg, and butter, letting the dough rise, preparing a cinnamon mixture, rolling out the dough with the cinnamon, cutting and'

In [54]:
pip install 'moto[s3,firehose,sqs]'

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
import moto

In [57]:
moto.mock_s3

AttributeError: module 'moto' has no attribute 'mock_s3'

In [58]:
from moto import mock_s3, mock_sqs, mock_firehose

ImportError: cannot import name 'mock_s3' from 'moto' (/Users/maximebonnesoeur/.pyenv/versions/bring_/lib/python3.10/site-packages/moto/__init__.py)

In [63]:
from moto import mock_aws
import boto3
import botocore.exceptions


# Testing with mock AWS services using moto
@mock_aws
def test_sqs_firehose_integration():
    # Create a mock S3 bucket
    s3 = boto3.resource("s3", region_name="us-east-1")
    bucket_name = "mock-bucket"
    s3.create_bucket(Bucket=bucket_name)

    # Create a mock SQS queue
    sqs = boto3.client("sqs", region_name="us-east-1")
    queue_name = "mock-queue"
    sqs_queue = sqs.create_queue(QueueName=queue_name)
    queue_url = sqs_queue["QueueUrl"]

    # Create a mock Firehose delivery stream
    firehose = boto3.client("firehose", region_name="us-east-1")
    firehose_stream_name = "mock-firehose"
    firehose.create_delivery_stream(
        DeliveryStreamName=firehose_stream_name,
        S3DestinationConfiguration={
            "RoleARN": "arn:aws:iam::123456789012:role/mock-firehose-role",
            "BucketARN": f"arn:aws:s3:::{bucket_name}",
            "Prefix": "firehose/",
        },
    )

    # Send a sample URL message to the SQS queue
    sample_url = (
        "https://www.marmiton.org/recettes/recette_cinnamon-rolls-de-karine_327467.aspx"
    )
    sqs.send_message(QueueUrl=queue_url, MessageBody=json.dumps({"url": sample_url}))

    # Receive message from SQS queue
    response = sqs.receive_message(QueueUrl=queue_url, MaxNumberOfMessages=1)

    messages = response.get("Messages", [])
    if messages:
        for message in messages:
            try:
                # Extract URL from the message body
                body = json.loads(message["Body"])
                url = body.get("url")
                if not url:
                    raise ValueError("No URL found in the message body.")

                # Extract recipe information
                extractor = RecipeExtractor(api_key="YOUR_OPENAI_API_KEY")
                recipe_info = extractor.extract_recipe_info(url)

                # Send the results to Firehose
                firehose.put_record(
                    DeliveryStreamName=firehose_stream_name,
                    Record={"Data": json.dumps(recipe_info) + "\n"},
                )

                # Delete the message from the queue after successful processing
                sqs.delete_message(
                    QueueUrl=queue_url, ReceiptHandle=message["ReceiptHandle"]
                )

            except Exception as e:
                print(f"Error processing message: {e}")

    # Validate that the data was sent to the mock S3 bucket
    s3_client = boto3.client("s3", region_name="us-east-1")
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix="firehose/")
    for obj in response.get("Contents", []):
        print(f"Found object in mock S3: {obj['Key']}")

In [64]:
test_sqs_firehose_integration()

RuntimeError: Credentials were refreshed, but the refreshed credentials are still expired.